# Enhanced Printer

### Extra exercise in chapter 4 ([link](https://runestone.academy/runestone/books/published/pythonds3/BasicDS/SimulationPrintingTasks.html))

A larger number of students definitely requires a different pages per minute speed. In particular, we can parameterize the number of students in the simulation, so that the actual number of students can be taken into account when choosing the pages per minute rate. This is the code before any modification.

In [2]:
import random
from pythonds3.basic import Queue


class Printer:
    def __init__(self, ppm):
        self.page_rate = ppm
        self.current_task = None
        self.time_remaining = 0

    def tick(self):
        if self.current_task is not None:
            self.time_remaining = self.time_remaining - 1
            if self.time_remaining <= 0:
                self.current_task = None

    def busy(self):
        return self.current_task is not None

    def start_next(self, new_task):
        self.current_task = new_task
        self.time_remaining = new_task.get_pages() * 60 / self.page_rate


class Task:
    def __init__(self, time):
        self.timestamp = time
        self.pages = random.randrange(1, 21)

    def get_stamp(self):
        return self.timestamp

    def get_pages(self):
        return self.pages

    def wait_time(self, current_time):
        return current_time - self.timestamp


def simulation(num_milliseconds, pages_per_minute):
    lab_printer = Printer(pages_per_minute)
    print_queue = Queue()
    waiting_times = []

    for current_millisecond in range(num_milliseconds):
        if new_print_task():
            task = Task(current_millisecond)
            print_queue.enqueue(task)

        if (not lab_printer.busy()) and (not print_queue.is_empty()):
            nexttask = print_queue.dequeue()
            waiting_times.append(nexttask.wait_time(current_millisecond))
            lab_printer.start_next(nexttask)

        lab_printer.tick()

    average_wait = sum(waiting_times) / len(waiting_times)
    print("Average Wait %6.2f millisecs %3d tasks remaining." % (average_wait, print_queue.size()))


def new_print_task():
    num = random.randrange(1, 181)
    return num == 180
    

The code with the proper modification is as shown below. Let's modify the `simulation()` method, by taking as a parameter the number of students rather than the speed.

In [20]:
MAX_SPEED = 20

def simulation(num_milliseconds, num_students):
    num_minutes = (num_milliseconds / 1000) / 60
    candidate_speed = num_students // num_minutes + 1 
    pages_per_minute = candidate_speed if candidate_speed <= MAX_SPEED else MAX_SPEED
    lab_printer = Printer(pages_per_minute)
    print_queue = Queue()
    waiting_times = []

    for current_millisecond in range(num_milliseconds):
        if new_print_task():
            task = Task(current_millisecond)
            print_queue.enqueue(task)

        if (not lab_printer.busy()) and (not print_queue.is_empty()):
            nexttask = print_queue.dequeue()
            waiting_times.append(nexttask.wait_time(current_millisecond))
            lab_printer.start_next(nexttask)

        lab_printer.tick()

    average_wait = sum(waiting_times) / len(waiting_times)
    print(f"Average Wait {average_wait:6.2f} millisecs {print_queue.size():3d} tasks remaining.")

In this way, we are selecting a speed that grows with the number of students but at the same time decreases with the number of milliseconds (since this is the time set to finish the simulation), while always being lower than the maximum speed set by an hypothetical printer (it usually takes a little more than one second). Let us try a simulation now, with an increasing number of students.

In [29]:
for i in range(1, 11):
    simulation(3600, i)

Average Wait   4.41 millisecs   0 tasks remaining.
Average Wait   1.76 millisecs   0 tasks remaining.
Average Wait   1.38 millisecs   0 tasks remaining.
Average Wait   2.96 millisecs   0 tasks remaining.
Average Wait   1.86 millisecs   0 tasks remaining.
Average Wait   1.90 millisecs   0 tasks remaining.
Average Wait   4.21 millisecs   0 tasks remaining.
Average Wait   8.23 millisecs   0 tasks remaining.
Average Wait   5.05 millisecs   2 tasks remaining.
Average Wait   1.08 millisecs   0 tasks remaining.
